***Model Training For Symptom to Disease Prediction***

**Loading the dataset**

In [2]:
import pandas as pd

dataset1 = pd.read_csv('datasets/Symptom2Disease.csv')
dataset1.head()

,Unnamed: 0,label,text
0,0,Psoriasis,I have been experiencing a skin rash on my arm...
1,1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,2,Psoriasis,I have been experiencing joint pain in my fing...
3,3,Psoriasis,"There is a silver like dusting on my skin, esp..."
4,4,Psoriasis,"My nails have small dents or pits in them, and..."


In [3]:
dataset2 = pd.read_csv('datasets/Diseases_Symptoms.csv')
dataset2.head()

,Code,Name,Symptoms,Treatments
0,1,Panic disorder,"Palpitations, Sweating, Trembling, Shortness o...","Antidepressant medications, Cognitive Behavior..."
1,2,Vocal cord polyp,"Hoarseness, Vocal Changes, Vocal Fatigue","Voice Rest, Speech Therapy, Surgical Removal"
2,3,Turner syndrome,"Short stature, Gonadal dysgenesis, Webbed neck...","Growth hormone therapy, Estrogen replacement t..."
3,4,Cryptorchidism,"Absence or undescended testicle(s), empty scro...",Observation and monitoring (in cases of mild o...
4,5,Ethylene glycol poisoning-1,"Nausea, vomiting, abdominal pain, General mala...","Supportive Measures, Gastric Decontamination, ..."


In [4]:
dataset3 = pd.read_csv('datasets/dataset.csv')
dataset3.head()

,disease,symptoms,cures,doctor,risk level
0,flu,"fever,cough,sore throat,runny or stuffy nose,m...","over-the-counter medications,rest,fluids","family doctor,urgent care",low (0.1%
1,bronchitis,"cough,mucus production,shortness of breath,che...","antibiotics,over-the-counter medications,rest,...","family doctor,pulmonologist",low (0.5%
2,pneumonia,"fever,cough,shortness of breath,chest pain,fat...","antibiotics,over-the-counter medications,rest,...","family doctor,pulmonologist",moderate (1%)
3,heart attack,"chest pain,shortness of breath,nausea,vomiting...",emergency medical services,cardiologist,high (20%)
4,stroke,"sudden weakness,numbness on one side of the bo...",emergency medical services,neurologist,high (15%)


In [5]:
#dropping unnecessary columns and merging the two datasets

#dropping Treatment column from d2
dataset2 = dataset2.drop(columns=['Treatments'])
dataset3 = dataset3.drop(columns=['cures', 'doctor', 'risk level'])
# Rename columns for clarity
dataset1 = dataset1.rename(columns={'label': 'Disease', 'text': 'Symptoms'})
dataset2 = dataset2.rename(columns={'Name': 'Disease'})
dataset3 = dataset3.rename(columns={'disease': 'Disease', 'symptoms': 'Symptoms'})

# Combine the datasets
dataset = pd.concat([dataset1[['Disease', 'Symptoms']], dataset2[['Disease', 'Symptoms']], dataset3[['Disease', 'Symptoms']]], ignore_index=True)
dataset.head()

,Disease,Symptoms
0,Psoriasis,I have been experiencing a skin rash on my arm...
1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,Psoriasis,I have been experiencing joint pain in my fing...
3,Psoriasis,"There is a silver like dusting on my skin, esp..."
4,Psoriasis,"My nails have small dents or pits in them, and..."


In [6]:
print(dataset.shape)

(1699, 2)


**Analyzing the Dataset**

In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1699 entries, 0 to 1698
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Disease   1699 non-null   object
 1   Symptoms  1699 non-null   object
dtypes: object(2)
memory usage: 26.7+ KB


In [8]:
# Check for missing values
missing_values = dataset.isnull().sum()
missing_values

Disease     0
Symptoms    0
dtype: int64

In [9]:
dataset.describe().T

,count,unique,top,freq
Disease,1699,498,diabetes,52
Symptoms,1699,1627,"I've been feeling extremely scratchy, sick, an...",4


In [10]:
dataset['Disease'].value_counts()

Disease
diabetes                          52
Psoriasis                         51
Migraine                          51
Pneumonia                         51
Arthritis                         51
                                  ..
Hypothyroidism                     1
Otosclerosis                       1
Complex Regional Pain Syndrome     1
Conversion Disorder                1
lupus                              1
Name: count, Length: 498, dtype: int64

In [16]:
import string
def remove_punctuation(text):
    if isinstance(text,str):
        return text.translate(str.maketrans('','',string.punctuation))
    return text
def to_lowercase(text):
    if isinstance(text, str):
        return text.lower()
    return text
dataset['Symptoms'] = dataset['Symptoms'].apply(remove_punctuation)

dataset['Symptoms'] = dataset['Symptoms'].apply(to_lowercase)


In [20]:
import spacy
nlp = spacy.load("en_core_sci_md")
def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text,ent.label_)for ent in doc.ents]
    return entities

dataset['Entities'] = dataset['Symptoms'].apply(extract_entities)
dataset.head()

OSError: [E050] Can't find model 'en_core_sci_md'. It doesn't seem to be a Python package or a valid path to a data directory.